In [1]:
import pandas as pd
import numpy
import math

In [2]:
fulldata = pd.read_csv('Data\yonelimfinal.csv').drop(columns=['Timestamp', 'Bolge', 'Yas', 'Egitim'])
data = fulldata.iloc[461:471]
data

,Cinsiyet,soru1,soru2,soru3,soru4,soru5,soru6,soru7,soru8,soru9,soru10,parti
461,Erkek,Hayır,Evet,Evet,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,DIĞER
462,Erkek,Hayır,Evet,Hayır,Evet,Hayır,Hayır,Evet,Hayır,Evet,Evet,IYI PARTI
463,Erkek,Evet,Evet,Evet,Evet,Hayır,Hayır,Evet,Evet,Evet,Evet,IYI PARTI
464,Erkek,Evet,Evet,Hayır,Hayır,Hayır,Hayır,Hayır,Hayır,Hayır,Evet,MHP
465,Erkek,Evet,Evet,Evet,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,IYI PARTI
466,Erkek,Hayır,Evet,Evet,Hayır,Hayır,Hayır,Evet,Evet,Evet,Evet,CHP
467,Erkek,Hayır,Evet,Hayır,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Hayır,CHP
468,Erkek,Hayır,Evet,Hayır,Evet,Hayır,Hayır,Evet,Evet,Evet,Hayır,CHP
469,Erkek,Hayır,Evet,Evet,Evet,Hayır,Hayır,Evet,Hayır,Hayır,Hayır,DIĞER
470,Erkek,Hayır,Evet,Hayır,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,DIĞER


In [3]:
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = pd.value_counts(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            # use log from math and set base to 2
            entropy += prob * math.log(prob, 2)
    
    return -entropy

In [4]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    #Find the unique values in the column
    values = data[split_name].unique()
    

    # Make two subsets of the data, based on the unique values
    left_split = data[data[split_name] == values[0]]
    right_split = data[data[split_name] == values[1]]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

In [5]:
columns = ['Cinsiyet', 'soru1', 'soru2', 'soru3', 'soru4', 'soru5', 
           'soru6', 'soru7', 'soru8', 'soru9', 'soru10']

In [6]:
def highest_info_gain(data, columns):
  #Intialize an empty dictionary for information gains
  information_gains = {}
  
  #Iterate through each column name in our list
  for col in columns:
    if(data[col].unique().size < 2):
        continue
    #Find the information gain for the column
    information_gain = calc_information_gain(data, col, 'parti')
    #Add the information gain to our dictionary using the column name as the key                                         
    information_gains[col] = information_gain
  
  #Return the key with the highest value                                          
  return max(information_gains, key=information_gains.get)

In [7]:
print(highest_info_gain(data, columns))

soru1


In [8]:
lc = data[data['soru1'] == 'Evet'].drop(columns=['soru1'])
lc

,Cinsiyet,soru2,soru3,soru4,soru5,soru6,soru7,soru8,soru9,soru10,parti
463,Erkek,Evet,Evet,Evet,Hayır,Hayır,Evet,Evet,Evet,Evet,IYI PARTI
464,Erkek,Evet,Hayır,Hayır,Hayır,Hayır,Hayır,Hayır,Hayır,Evet,MHP
465,Erkek,Evet,Evet,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,IYI PARTI


In [9]:
rc = data[data['soru1'] == 'Hayır'].drop(columns=['soru1'])
rc

,Cinsiyet,soru2,soru3,soru4,soru5,soru6,soru7,soru8,soru9,soru10,parti
461,Erkek,Evet,Evet,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,DIĞER
462,Erkek,Evet,Hayır,Evet,Hayır,Hayır,Evet,Hayır,Evet,Evet,IYI PARTI
466,Erkek,Evet,Evet,Hayır,Hayır,Hayır,Evet,Evet,Evet,Evet,CHP
467,Erkek,Evet,Hayır,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Hayır,CHP
468,Erkek,Evet,Hayır,Evet,Hayır,Hayır,Evet,Evet,Evet,Hayır,CHP
469,Erkek,Evet,Evet,Evet,Hayır,Hayır,Evet,Hayır,Hayır,Hayır,DIĞER
470,Erkek,Evet,Hayır,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,DIĞER


In [10]:
cols = [y for y in columns if y != 'soru1']
print(highest_info_gain(lc, cols))

soru3


In [11]:
llc = lc[lc['soru3'] == 'Evet'].drop(columns=['soru3'])
llc

,Cinsiyet,soru2,soru4,soru5,soru6,soru7,soru8,soru9,soru10,parti
463,Erkek,Evet,Evet,Hayır,Hayır,Evet,Evet,Evet,Evet,IYI PARTI
465,Erkek,Evet,Hayır,Hayır,Hayır,Evet,Hayır,Evet,Evet,IYI PARTI


In [12]:
lrc = lc[lc['soru3'] == 'Hayır'].drop(columns=['soru3'])
lrc

,Cinsiyet,soru2,soru4,soru5,soru6,soru7,soru8,soru9,soru10,parti
464,Erkek,Evet,Hayır,Hayır,Hayır,Hayır,Hayır,Hayır,Evet,MHP


In [13]:
print(highest_info_gain(rc, cols))

soru8


In [14]:
rlc = rc[rc['soru8'] == 'Evet'].drop(columns=['soru8'])
rlc

,Cinsiyet,soru2,soru3,soru4,soru5,soru6,soru7,soru9,soru10,parti
466,Erkek,Evet,Evet,Hayır,Hayır,Hayır,Evet,Evet,Evet,CHP
468,Erkek,Evet,Hayır,Evet,Hayır,Hayır,Evet,Evet,Hayır,CHP


In [15]:
rrc = rc[rc['soru8'] == 'Hayır'].drop(columns=['soru8'])
rrc

,Cinsiyet,soru2,soru3,soru4,soru5,soru6,soru7,soru9,soru10,parti
461,Erkek,Evet,Evet,Hayır,Hayır,Hayır,Evet,Evet,Evet,DIĞER
462,Erkek,Evet,Hayır,Evet,Hayır,Hayır,Evet,Evet,Evet,IYI PARTI
467,Erkek,Evet,Hayır,Hayır,Hayır,Hayır,Evet,Evet,Hayır,CHP
469,Erkek,Evet,Evet,Evet,Hayır,Hayır,Evet,Hayır,Hayır,DIĞER
470,Erkek,Evet,Hayır,Hayır,Hayır,Hayır,Evet,Evet,Evet,DIĞER


In [16]:
cols2 = [y for y in cols if y != 'soru8']
print(highest_info_gain(rrc, cols2))

soru3


In [17]:
rrlc = rrc[rrc['soru3'] == 'Evet'].drop(columns=['soru3'])
rrlc

,Cinsiyet,soru2,soru4,soru5,soru6,soru7,soru9,soru10,parti
461,Erkek,Evet,Hayır,Hayır,Hayır,Evet,Evet,Evet,DIĞER
469,Erkek,Evet,Evet,Hayır,Hayır,Evet,Hayır,Hayır,DIĞER


In [18]:
rrrc = rrc[rrc['soru3'] == 'Hayır'].drop(columns=['soru3'])
rrrc

,Cinsiyet,soru2,soru4,soru5,soru6,soru7,soru9,soru10,parti
462,Erkek,Evet,Evet,Hayır,Hayır,Evet,Evet,Evet,IYI PARTI
467,Erkek,Evet,Hayır,Hayır,Hayır,Evet,Evet,Hayır,CHP
470,Erkek,Evet,Hayır,Hayır,Hayır,Evet,Evet,Evet,DIĞER


In [19]:
cols3 = [y for y in cols2 if y != 'soru3']
print(highest_info_gain(rrrc, cols3))

soru4


In [20]:
rrrlc = rrrc[rrrc['soru4'] == 'Evet'].drop(columns=['soru4'])
rrrlc

,Cinsiyet,soru2,soru5,soru6,soru7,soru9,soru10,parti
462,Erkek,Evet,Hayır,Hayır,Evet,Evet,Evet,IYI PARTI


In [21]:
rrrrc = rrrc[rrrc['soru4'] == 'Hayır'].drop(columns=['soru4'])
rrrrc

,Cinsiyet,soru2,soru5,soru6,soru7,soru9,soru10,parti
467,Erkek,Evet,Hayır,Hayır,Evet,Evet,Hayır,CHP
470,Erkek,Evet,Hayır,Hayır,Evet,Evet,Evet,DIĞER


In [22]:
cols4 = [y for y in cols3 if y != 'soru4']
print(highest_info_gain(rrrrc, cols4))

soru10


In [23]:
rrrrlc = rrrrc[rrrrc['soru10'] == 'Evet'].drop(columns=['soru10'])
rrrrlc

,Cinsiyet,soru2,soru5,soru6,soru7,soru9,parti
470,Erkek,Evet,Hayır,Hayır,Evet,Evet,DIĞER


In [24]:
rrrrrc = rrrrc[rrrrc['soru10'] == 'Hayır'].drop(columns=['soru10'])
rrrrrc

,Cinsiyet,soru2,soru5,soru6,soru7,soru9,parti
467,Erkek,Evet,Hayır,Hayır,Evet,Evet,CHP


In [25]:
def id3Check(record):
    if record['soru1'] == 'Evet':
        if record['soru3'] == 'Evet':
            return 'IYI PARTI'
        if record['soru3'] == 'Hayır':
            return 'MHP'
    if record['soru1'] == 'Hayır':
        if record['soru8'] == 'Evet':
            return 'CHP'
        if record['soru8'] == 'Hayır':
            if record['soru3'] == 'Evet':
                return 'DIĞER'
            if record['soru3'] == 'Hayır':
                if record['soru4'] == 'Evet':
                    return 'IYI PARTI'
                if record['soru4'] == 'Hayır':
                    if record['soru10'] == 'Evet':
                        return 'DIĞER'
                    if record['soru10'] == 'Hayır':
                        return 'CHP'

In [26]:
correct = 0
for i in range(885):
    if id3Check(fulldata.iloc[i]) == fulldata.iloc[i]['parti']:
        correct += 1
round(correct/885, 3)

0.183

In [27]:
from chefboost import Chefboost as chef

df = data.rename(columns={'parti': 'Decision'})

config = {'algorithm': 'ID3'}
model = chef.fit(df, config = config)

ID3  tree is going to be built...
-------------------------
finished in  0.803849458694458  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  100.0 % on  10  instances
Labels:  ['DIĞER' 'IYI PARTI' 'MHP' 'CHP']
Confusion matrix:  [[3, 0, 0, 0], [0, 3, 0, 0], [0, 0, 1, 0], [0, 0, 0, 3]]
Decision  DIĞER  => Accuray:  100.0 %, Precision:  100.0 %, Recall:  100.0 %, F1:  100.0 %
Decision  IYI PARTI  => Accuray:  100.0 %, Precision:  100.0 %, Recall:  100.0 %, F1:  100.0 %
Decision  MHP  => Accuray:  100.0 %, Precision:  100.0 %, Recall:  100.0 %, F1:  100.0 %
Decision  CHP  => Accuray:  100.0 %, Precision:  100.0 %, Recall:  100.0 %, F1:  100.0 %


In [28]:
from chefboost import Chefboost as chef

df = fulldata.rename(columns={'parti': 'Decision'}).sample(300)

config = {'algorithm': 'ID3'}
model = chef.fit(df, config = config)

ID3  tree is going to be built...
-------------------------
finished in  22.943624019622803  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  70.66666666666667 % on  300  instances
Labels:  ['IYI PARTI' 'CHP' 'MHP' 'DIĞER' 'AKP' 'HDP']
Confusion matrix:  [[71, 21, 2, 11, 1, 1], [23, 54, 1, 14, 1, 3], [2, 2, 8, 0, 0, 0], [0, 1, 0, 36, 1, 0], [1, 0, 0, 3, 40, 0], [0, 0, 0, 0, 0, 3]]
Decision  IYI PARTI  => Accuray:  79.3333 %, Precision:  66.3551 %, Recall:  73.1959 %, F1:  69.6078 %
Decision  CHP  => Accuray:  78.0 %, Precision:  56.25 %, Recall:  69.2308 %, F1:  62.069 %
Decision  MHP  => Accuray:  97.6667 %, Precision:  66.6667 %, Recall:  72.7273 %, F1:  69.5652 %
Decision  DIĞER  => Accuray:  90.0 %, Precision:  94.7368 %, Recall:  56.25 %, F1:  70.5882 %
Decision  AKP  => Accuray:  97.6667 %, Precision:  90.9091 %, Recall:  93.0233 %, F1:  91.954 %
Decision  HDP  => Accuray:  98.6667 %, Precision:  100.0 %, Recall:  42.8571 %, F1:  60.0 %


In [29]:
def id3Check(record):
    cinsiyet = record['Cinsiyet']
#    Yas = record['Yas']
#    Bolge = record['Bolge']
#    Egitim = record['Egitim']
    soru1 = record['soru1']
    soru2 = record['soru2']
    soru3 = record['soru3']
    soru4 = record['soru4']
    soru5 = record['soru5']
    soru6 = record['soru6']
    soru7 = record['soru7']
    soru8 = record['soru8']
    soru9 = record['soru9']
    soru10 = record['soru10']
    prediction = chef.predict(model, param = [cinsiyet, soru1,soru2,soru3,soru4,soru5,soru6,soru7,soru8,soru9,soru10])
    return prediction

In [30]:
correct = 0
for i in range(885):
    if id3Check(fulldata.iloc[i]) == fulldata.iloc[i]['parti']:
        correct += 1
round(correct/885, 3)

0.516

In [31]:
config2 = {'algorithm': 'C4.5'}
model2 = chef.fit(df, config = config2)

C4.5  tree is going to be built...
-------------------------
finished in  20.09727168083191  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  70.66666666666667 % on  300  instances
Labels:  ['IYI PARTI' 'CHP' 'MHP' 'DIĞER' 'AKP' 'HDP']
Confusion matrix:  [[71, 21, 2, 11, 1, 1], [23, 54, 1, 14, 1, 3], [2, 2, 8, 0, 0, 0], [0, 1, 0, 36, 1, 0], [1, 0, 0, 3, 40, 0], [0, 0, 0, 0, 0, 3]]
Decision  IYI PARTI  => Accuray:  79.3333 %, Precision:  66.3551 %, Recall:  73.1959 %, F1:  69.6078 %
Decision  CHP  => Accuray:  78.0 %, Precision:  56.25 %, Recall:  69.2308 %, F1:  62.069 %
Decision  MHP  => Accuray:  97.6667 %, Precision:  66.6667 %, Recall:  72.7273 %, F1:  69.5652 %
Decision  DIĞER  => Accuray:  90.0 %, Precision:  94.7368 %, Recall:  56.25 %, F1:  70.5882 %
Decision  AKP  => Accuray:  97.6667 %, Precision:  90.9091 %, Recall:  93.0233 %, F1:  91.954 %
Decision  HDP  => Accuray:  98.6667 %, Precision:  100.0 %, Recall:  42.8571 %, F1:  60.0 %


In [32]:
def c45Check(record):
    cinsiyet = record['Cinsiyet']
#    Yas = record['Yas']
#    Bolge = record['Bolge']
#    Egitim = record['Egitim']
    soru1 = record['soru1']
    soru2 = record['soru2']
    soru3 = record['soru3']
    soru4 = record['soru4']
    soru5 = record['soru5']
    soru6 = record['soru6']
    soru7 = record['soru7']
    soru8 = record['soru8']
    soru9 = record['soru9']
    soru10 = record['soru10']
    prediction = chef.predict(model2, param = [cinsiyet, soru1,soru2,soru3,soru4,soru5,soru6,soru7,soru8,soru9,soru10])
    return prediction

In [33]:
correct = 0
for i in range(885):
    if c45Check(fulldata.iloc[i]) == fulldata.iloc[i]['parti']:
        correct += 1
round(correct/885, 3)

0.499